# Data Wrangling Project
> the region selected was 'bengaluru, India' which is my hometown. The dataset is as large as 600+MB. And there were a few challenges I had to face while cleaning the data, which is described below.

In [1]:
# Explaination at the bottom

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import sqlite3
import cerberus
import schema

In [2]:
OSM_PATH = "bengaluru_india.osm"
NODES_PATH = "bangalore/node.csv"
NODE_TAGS_PATH = "bangalore/node_tags.csv"
WAYS_PATH = "bangalore/way.csv"
WAY_NODES_PATH = "bangalore/way_nodes.csv"
WAY_TAGS_PATH = "bangalore/way_tags.csv"


In [3]:
# VALIDATION PROCESS

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

## Fixing street names
> Here in bangalore we refer to roads using 'Main' and 'Cross' most of the time. Road names might be abbreviated to 'Rd' but you find very few 'Street' or 'Lane'
> Secondly Some of the data might have the area name written. This is not the road name. So a data could be something like 
>> '17th Main, BTM Layout' But we need to change it to '17th Main Road'.We got to consider 2 things. 
>>> - If Road is missing from any value that has 'Main', or 'Cross', we add 'Road' <br/>
>>> - Any value after road is removed.

> Sometime the area name is given after the road name. That also is avoided in the code below


In [35]:
def value_fixer(value):
#     Make all upper/lower case of 'road|cross|main' to Road
    re_road = re.compile(re.escape('road'), re.IGNORECASE)
    re_main = re.compile(re.escape('main'), re.IGNORECASE)
    re_cross = re.compile(re.escape('cross'), re.IGNORECASE)
    if re.search('road', value, re.IGNORECASE):
        value = re_road.sub('Road',value)
    if re.search('cross', value, re.IGNORECASE):
        value = re_road.sub('Cross',value)
    if re.search('main', value, re.IGNORECASE):
        value = re_road.sub('Main',value)

#     Fix Rd or Rd. to Road
#     Added a space so as to avoid considering 'rd' as a part of '3rd' and only taking abc rd.
    re_rd_period = re.compile(re.escape(' rd\.'), re.IGNORECASE)
    re_rd = re.compile(re.escape(' rd'), re.IGNORECASE)
    if re.search(' rd\.', value, re.IGNORECASE):
        value = re_rd_period.sub(' Road', value)
    elif re.search(' rd', value, re.IGNORECASE):
        value = re_rd.sub(' Road', value)


#     Fix mn or mn. to Main
    re_mn_period = re.compile(re.escape('mn.'), re.IGNORECASE)
    re_mn = re.compile(re.escape('mn'), re.IGNORECASE)
    if 'Main' not in value:
        if re.search('th mn\.', value, re.IGNORECASE) or re.search('st mn\.', value, re.IGNORECASE) or re.search('nd mn\.', value, re.IGNORECASE) or re.search('rd mn\.', value, re.IGNORECASE):
            value = re_mn_period.sub('Main', value)
        elif re.search('th mn', value, re.IGNORECASE) or re.search('st mn', value, re.IGNORECASE) or re.search('nd mn', value, re.IGNORECASE) or re.search('rd mn', value, re.IGNORECASE):
            value = re_mn.sub('Main', value)
    if 'Main Road' not in value:
        value = value.replace('Main','Main Road')

#     Fix crs or Crs. or cros to Cross
    re_crs_period = re.compile(re.escape('crs.'), re.IGNORECASE)
    re_crs = re.compile(re.escape('crs'), re.IGNORECASE)
    re_cros = re.compile(re.escape('cros'), re.IGNORECASE)
    if 'Cross' not in value:
        if re.search('th crs\.', value, re.IGNORECASE) or re.search('st crs\.', value, re.IGNORECASE) or re.search('nd crs\.', value, re.IGNORECASE) or re.search('rd crs\.', value, re.IGNORECASE):
            value = re_crs_period.sub('Cross', value)
        elif re.search('th crs', value, re.IGNORECASE) or re.search('st crs', value, re.IGNORECASE) or re.search('nd crs', value, re.IGNORECASE) or re.search('rd crs', value, re.IGNORECASE):
            value = re_crs.sub('Cross', value)
        elif re.search('cros', value, re.IGNORECASE):
            value = re_crs.sub('Cross', value)
    if 'Cross Road' not in value:
        value = value.replace('Cross','Cross Road')

#     Remove everything after 'Road'. I.e. Removing area name for roads. Like 4th Main Road, Banashankri should become 4th Main Road.
    if 'Road' in value:
        value=value[0:value.index('Road')+4]
    return value




## Key values that are not so good

> 1. Some nodes value are given as 'traffic_signals' or 'bus_stop' It would be better if they didn't have _ seperating them. so maybe replace '_' with ' '

> 2. Finally. Doing 1st step, might create an issue if we are changing a user name. User name eg. 'my_name' will change to 'my name' which shouldn't happen. So give an exception of changing value of key: 'created_by'

> 3. Some key:'source' have value:'AND' -> Not sure what it refers to. But they appear a few times. Not sure if they need to be removed or not.

In [42]:
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    count = 0
    count2 = 0
    if element.tag == 'node':
        node_attribs= {
            'id': int(element.attrib['id']),
            'user': element.attrib['user'].encode('utf-8').strip(),
            'uid': int(element.attrib['uid']),
            'version': element.attrib['version'].encode('utf-8').strip(),
            'lat': float(element.attrib['lat']),
            'lon': float(element.attrib['lon']),
            'timestamp': element.attrib['timestamp'].encode('utf-8').strip(),
            'changeset': int(element.attrib['changeset']),
        }
        for t in element:
            if t.tag=='tag':
#                 Pt. 1 and 2 of 'Key values that are not so good'
                if t.attrib['k'] is not 'created_by':
                    t.attrib['v'].replace('_',' ')
#                 We don't want to make name as :kn and type as 'name', so we make an alternative condition.
                if ':kn' not in t.attrib['k']:
                    if ':' in t.attrib['k']:
                        type = t.attrib['k'][0:t.attrib['k'].index(':')]
                        key = t.attrib['k'][t.attrib['k'].index(':')+1:]
                    else:
                        type = 'regular'
                        key = t.attrib['k']
                    value = value_fixer(t.attrib['v'].encode('utf-8').strip())
                    tags.append({
                        'id':int(element.attrib['id']),
                        'key': key.encode('utf-8').strip(),
                        'value': value,
                        'type': type.encode('utf-8').strip()
                        })
                else:
                    if ':' in t.attrib['k'][:t.attrib['k'].index(':kn')]:
                        type = t.attrib['k'][0:t.attrib['k'].index(':')]
                        key = t.attrib['k'][t.attrib['k'].index(':')+1:]
                    else:
                        type = 'regular'
                        key = t.attrib['k']
                    value = value_fixer(t.attrib['v'].encode('utf-8').strip())
                    tags.append({
                        'id':int(element.attrib['id']),
                        'key': key.encode('utf-8').strip(),
                        'value': value,
                        'type': type.encode('utf-8').strip()
                        })
        result = {'node': node_attribs, 'node_tags': tags}
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        way_attribs= {
            'id': int(element.attrib['id']),
            'user': element.attrib['user'].encode('utf-8').strip(),
            'uid': int(element.attrib['uid']),
            'version': element.attrib['version'].encode('utf-8').strip(),
            'timestamp': str(element.attrib['timestamp']),
            'changeset': int(element.attrib['changeset']),
        }
        n=0
        for t in element:
            if t.tag=='tag':
#                 Pt. 1 and 2 of 'Key values that are not so good'
                if t.attrib['k'] is not 'created_by':
                    t.attrib['v'].replace('_',' ')
                if ':kn' not in t.attrib['k']:
                    if ':' in t.attrib['k']:
                        type = t.attrib['k'][0:t.attrib['k'].index(':')]
                        key = t.attrib['k'][t.attrib['k'].index(':')+1:]
                    else:
                        type = 'regular'
                        key = t.attrib['k']
                    value = value_fixer(t.attrib['v'].encode('utf-8').strip())
                    tags.append({
                        'id':int(element.attrib['id']),
                        'key': key.encode('utf-8').strip(),
                        'value': value,
                        'type': type.encode('utf-8').strip()
                        })
                else:
#                     If tag is of local language kannada, then :kn must not be in the key. 
                    if ':' in t.attrib['k'][:t.attrib['k'].index(':kn')]:
                        print 'tag: ', t
                        print t.attrib['k']
                        print "format: type:key:language 2"
                        print t.attrib['k']
                        type = t.attrib['k'][0:t.attrib['k'].index(':')]
                        key = t.attrib['k'][t.attrib['k'].index(':')+1:]
                        print 'type:', type
                        print 'key:', key
                        print 'value: ', value_fixer(t.attrib['v'].encode('utf-8').strip())
                    else:
                        print 'tag: ', t
                        print t.attrib['k']
                        print "format key:language 2"
                        print t.attrib['k']
                        type = 'regular'
                        key = t.attrib['k']
                        print 'type:', type
                        print 'key:', key
                        print 'value: ', value_fixer(t.attrib['v'].encode('utf-8').strip())
                    value = value_fixer(t.attrib['v'].encode('utf-8').strip())
                    tags.append({
                        'id':int(element.attrib['id']),
                        'key': key.encode('utf-8').strip(),
                        'value': value,
                        'type': type.encode('utf-8').strip()
                        })
            elif t.tag=='nd':
                way_nodes.append({
                    'id': element.attrib['id'],
                    'node_id': t.attrib['ref'],
                    'position': n
                })
                n += 1
                
        result = {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

In [6]:
# USED CODE FROM THE FINAL CHAPTER OF THE DATA WRANGLING COURSE
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

In [7]:
# USED CODE FROM THE FINAL CHAPTER OF THE DATA WRANGLING COURSE
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        raise Exception(message_string.format(field, error_string))

In [8]:
# USED CODE FROM THE FINAL CHAPTER OF THE DATA WRANGLING COURSE

# ================================================== #
#               Helper Functions                     #
# ================================================== #
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [9]:
# USED CODE FROM THE FINAL CHAPTER OF THE DATA WRANGLING COURSE
# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [43]:
process_map(OSM_PATH, validate=False)

tag:  <Element 'tag' at 0x10412c930>
name:kn
format key:language 2
name:kn
type: regular
key: name:kn
value:  ಮಹಾತ್ಮ ಗಾಂಧಿ ರಸ್ತೆ
tag:  <Element 'tag' at 0x1040d8240>
name:kn
format key:language 2
name:kn
type: regular
key: name:kn
value:  ಕುಮಾರ ಕೃಪ ರಸ್ತೆ
tag:  <Element 'tag' at 0x10408f450>
name:kn
format key:language 2
name:kn
type: regular
key: name:kn
value:  ೩ನೇ ಮುಖ್ಯ ರಸ್ತೆ
tag:  <Element 'tag' at 0x10408f1e0>
name:kn
format key:language 2
name:kn
type: regular
key: name:kn
value:  ೨ನೇ ಮುಖ್ಯ ರಸ್ತೆ
tag:  <Element 'tag' at 0x1040d5a50>
name:kn
format key:language 2
name:kn
type: regular
key: name:kn
value:  ವಿಕ್ಟೊರಿಯ ರಸ್ತೆ
tag:  <Element 'tag' at 0x1040d5fc0>
name:kn
format key:language 2
name:kn
type: regular
key: name:kn
value:  ಲಾಲಬಾಗ್ ರಸ್ತೆ
tag:  <Element 'tag' at 0x1045e0b70>
name:kn
format key:language 2
name:kn
type: regular
key: name:kn
value:  ಕೆಂಗೇರಿ ಮುಖ್ಯ ರಸ್ತೆ
tag:  <Element 'tag' at 0x1045ddf00>
name:kn
format key:language 2
name:kn
type: regular
key: name:kn
value:  ಕೆಂ

# PUSH csv to SQLITE DB

In [11]:
# Creates SQL Table
def create_table(create_table_query,conn):
    c = conn.cursor()
    c.execute(create_table_query)
    conn.commit()

In [12]:
# Reads csv
def read_csv(csv_file):
    with open(csv_file, "rb") as f:
        reader = csv.reader(f)
        header = reader.next()
        data = [row for row in reader]
        return header,data

In [13]:
# adds each row of csv into sql table
def add_into_table(csv_file):
    c = conn.cursor()
    table_name = csv_file[csv_file.index('/')+1:csv_file.index('.csv')]
    header,data = read_csv(csv_file)
    header_string = '(\"'+ '\", \"'.join(str(x) for x in header)+'\")'
    for row in data:
        data_string = '(\"'+ '\", \"'.join(str(x).replace('\"','').replace('\'','') for x in row)+'\")'
        query_string = "INSERT INTO "+table_name+header_string+" VALUES "+data_string
        try:
            c.execute(query_string)
        except:
            print 'error query: ', query_string
            conn.commit()
            return
    conn.commit()

In [20]:
conn = sqlite3.connect('bengaluru_map.db')
# Create tables for each csv file
# Node
create_table_query = '''CREATE TABLE IF NOT EXISTS node(id INTEGER, lat REAL, lon REAL, user STRING, uid INTEGER,
                    version STRING, changeset INTEGER, timestamp STRING, PRIMARY KEY(id ASC))'''
create_table(create_table_query,conn)

# Node_tags
conn = sqlite3.connect('bengaluru_map.db')
create_table_query = '''CREATE TABLE IF NOT EXISTS node_tags(id INTEGER, key STRING, value STRING, type STRING,
                    FOREIGN KEY(id) REFERENCES node(id))'''
create_table(create_table_query,conn)
# way
create_table_query = '''CREATE TABLE IF NOT EXISTS way(id INTEGER, user STRING, uid INTEGER, version STRING,
                    changeset INTEGER, timestamp STRING, PRIMARY KEY(id ASC))'''
create_table(create_table_query,conn)

# way_nodes
create_table_query = '''CREATE TABLE IF NOT EXISTS way_nodes(id INTEGER, node_id INTEGER, position INTEGER, 
                    FOREIGN KEY(id) REFERENCES way(id), FOREIGN KEY(node_id) REFERENCES node(id))'''
create_table(create_table_query,conn)

# way_tags
create_table_query = '''CREATE TABLE IF NOT EXISTS way_tags(id INTEGER, key STRING, value STRING, type STRING,
                    FOREIGN KEY(id) REFERENCES way(id))'''
create_table(create_table_query,conn)

conn.close()

In [15]:
# To drop table incase you want to try again
# conn = sqlite3.connect('bengaluru_map.db')
# drop_query = '''DROP TABLE node_tags'''
# create_table(drop_query,conn)

In [21]:
# Build connection
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
# Add data into table:
# Node
add_into_table("bangalore/node.csv")
# Node_tags
add_into_table("bangalore/node_tags.csv")
# way
add_into_table("bangalore/way.csv")
# way_nodes
add_into_table("bangalore/way_nodes.csv")
# way_tags
add_into_table("bangalore/way_tags.csv")
conn.close()

# QUERIES

### The number of rows that were written in 'Kannada', the local language: 

In [22]:
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute("SELECT COUNT(*) FROM node_tags WHERE key LIKE '%:kn%'")
for row in result:
    print row
conn.close()

(4444,)


In [23]:
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute("SELECT COUNT(*) FROM way_tags WHERE key LIKE '%:kn%'")
for row in result:
    print row
conn.close()

(8109,)


### The most popular amenities in Bangalore: 
> There are a lot of restaurants, cafes and fastfood joints around bangalore. So we can explore them and also the find the more popular banks

In [24]:
# USED CODE FROM THE SAMEPLE CODE GIVEN IN https://gist.github.com/carlward/54ec1c91b62a5f911c42#map-area
# Questions - Most popular Amenities in bangalore
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute("SELECT value,count(*) FROM node_tags WHERE key LIKE '%amenity%' GROUP BY value ORDER BY count(*) DESC LIMIT 10")
for row in result:
    print row
conn.close()

(u'restaurant', 1697)
(u'atm', 799)
(u'bank', 743)
(u'place_of_worship', 701)
(u'pharmacy', 553)
(u'fast_food', 515)
(u'hospital', 454)
(u'school', 371)
(u'cafe', 350)
(u'fuel', 282)


In [25]:
# MODIFIED CODE FROM THE SAMEPLE CODE GIVEN IN https://gist.github.com/carlward/54ec1c91b62a5f911c42#map-area
# Questions - Most popular Banks in India
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute("SELECT node_tags.value, COUNT(*) as num FROM node_tags,(SELECT DISTINCT(id) FROM node_tags WHERE value LIKE '%bank%') as banknodes ON node_tags.id=banknodes.id WHERE node_tags.key IN ('name','operator','brand') AND node_tags.value LIKE '%bank%' GROUP BY node_tags.value ORDER BY num DESC LIMIT 10")
for row in result:
    print row
conn.close()

(u'State Bank of India', 194)
(u'Canara Bank', 126)
(u'ICICI Bank', 105)
(u'Axis Bank', 104)
(u'HDFC Bank', 89)
(u'Corporation Bank', 48)
(u'Karnataka Bank', 48)
(u'Vijaya Bank', 42)
(u'Syndicate Bank', 31)
(u'Citibank', 30)


In [26]:
# USED CODE FROM THE SAMEPLE CODE GIVEN IN https://gist.github.com/carlward/54ec1c91b62a5f911c42#map-area
# Questions - Food in bangalore - The more popular cuisines
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute("SELECT value,count(*) as quantity FROM node_tags,(SELECT DISTINCT(id) FROM node_tags WHERE value IN ('restaurant','cafe','fast_food')) as foodnodes ON node_tags.id=foodnodes.id WHERE key IN ('cuisine') GROUP BY value ORDER BY quantity DESC LIMIT 10")
# u'cuisine'
# result=c.execute("SELECT DISTINCT(id) FROM node_tags WHERE value IN ('restaurant','cafe','fast_food')")
for row in result:
    print row
conn.close()

(u'regional', 368)
(u'indian', 292)
(u'pizza', 90)
(u'vegetarian', 89)
(u'chinese', 78)
(u'ice_cream', 52)
(u'coffee_shop', 50)
(u'burger', 45)
(u'international', 31)
(u'italian', 29)


In [27]:
# Number of roads in bangalore
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute("SELECT count(*) FROM way_tags WHERE value LIKE '%Road%'")
for row in result:
    print row
conn.close()

(10325,)


# Top 10 Node id where highest number of ways intersect 

In [28]:
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
node_through_which_highest_way_pass = "(SELECT node_id,count(*) FROM way_nodes GROUP BY node_id ORDER BY count(*) DESC LIMIT 10) as busy_nodes"
info_about_that_node = "SELECT id FROM node,"+node_through_which_highest_way_pass+" ON busy_nodes.node_id=node.id WHERE node.id=busy_nodes.node_id "
result = c.execute(info_about_that_node)
op=[]
for row in result:
    op.append(row[0])
conn.close()

In [29]:
print op

[3676386504, 3756817769, 3756817774, 3676386503, 3750785900, 3751654300, 3756817768, 3676374899, 3676374909, 3750785901]


In [30]:
# To find out more about these nodes, we investigated further. We find no node tags. 
tags_of_those_nodes = "SELECT * FROM node_tags,("+info_about_that_node+") as main_nodes WHERE main_nodes.id=node_tags.id"
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute(tags_of_those_nodes)
for row in result:
    print row
conn.close()

## Vidhan Souda
> There were no information on these nodes, so I went through google maps entering these lat,lons.
> Found something interesting. All these nodes are a part of or few minutes away from 'Vidhan Souda' which is the state legislature building of karnataka. Which makes sense. There might be a lot of ways - including the walking paths, buildings, walls, metro, roads that pass through it, roads that go around it, service roads, etc.

## Additional Statistics :
### Number of Rows
1. Number of nodes: 2882959
1. Number of node_tags: 93243
1. Number of ways: 660784
1. Number of way_nodes: 3576371
1. Number of way_tags: 723631

### Size of each CSV file it was retreived from
1. Size of node.csv file: 241.3 MB
1. Size of node_tags.csv file: 3.7 MB
1. Size of way.csv file: 40.3 MB
1. Size of way_nodes.csv file: 85.8 MB
1. Size of way_tags.csv file: 24.1 MB

In [31]:
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute("SELECT COUNT(*) FROM node")
for row in result:
    print row
conn.close()

(2882959,)


In [32]:
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute("SELECT COUNT(*) FROM node_tags")
for row in result:
    print row
conn.close()

(93243,)


In [33]:
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute("SELECT COUNT(*) FROM way")
for row in result:
    print row
conn.close()

(660784,)


In [34]:
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute("SELECT COUNT(*) FROM way_nodes")
for row in result:
    print row
conn.close()

(3576371,)


In [35]:
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute("SELECT COUNT(*) FROM way_tags")
for row in result:
    print row
conn.close()

(723631,)


In [36]:
# USED CODE FROM THE SAMEPLE CODE GIVEN IN https://gist.github.com/carlward/54ec1c91b62a5f911c42#map-area
# NUMBER OF UNIQUE USERS 
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute("SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM node UNION ALL SELECT uid FROM way) e")
for row in result:
    print row[0]
conn.close()

1999


In [37]:
# USED CODE FROM THE SAMEPLE CODE GIVEN IN https://gist.github.com/carlward/54ec1c91b62a5f911c42#map-area
# Highest contributing user
conn = sqlite3.connect('bengaluru_map.db')
c = conn.cursor()
result = c.execute("SELECT e.user, COUNT(*) as num FROM (SELECT user FROM node UNION ALL SELECT user FROM way) e GROUP BY e.user ORDER BY num DESC LIMIT 10")
for row in result:
    print row
conn.close()

(u'jasvinderkaur', 124900)
(u'akhilsai', 118687)
(u'premkumar', 115884)
(u'saikumar', 114996)
(u'shekarn', 98118)
(u'PlaneMad', 94732)
(u'vamshikrishna', 94275)
(u'himalay', 88246)
(u'himabindhu', 86844)
(u'sdivya', 84998)
